In [0]:
%pip install -r ../requirements.txt
%pip install opencv-python

In [0]:
dbutils.library.restartPython()

In [0]:
from dbx.pixels.dicom.redactor import Redactor

redactor = Redactor(spark=spark)

query = redactor.process_from_table(
  source_table='ema_rina.pixels_solacc_gehc.object_catalog_redaction',
  checkpoint_location='/Volumes/ema_rina/pixels_solacc_gehc/pixels_volume/_checkpoints/redactor',
  trigger_available_now=True
)

In [0]:
%sql
select * from ema_rina.pixels_solacc_gehc.object_catalog_redaction